In [1]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import sys
sys.path.append('../')
from src.graph_utilities import lcc_excluding_no_content

In [2]:
# with open('../data/Cit-HepPh.txt', 'r') as f:
#     for _ in range(4):
#         f.readline()
#     edges = []
#     while True:
#         line = f.readline()
#         if not line:
#             break
#         edges.append(line.split())
        
# # generate synthetic labels
# labels = ('agreement', 'neutral', 'disagreement')
# for i in range(len(edges)):
#     label = random.choice(labels)
#     edges[i].append(label)

# # generate synthetic embeddings
# nodes = set()
# for edge in edges:
#     nodes.add(edge[0])
#     nodes.add(edge[1])
# nodes = list(nodes)

# embeddings = {node: np.random.rand(10) for node in nodes}

# # convert to graph
# g = ig.Graph.TupleList(edges, directed=True, edge_attrs=['label'])
# g.vs['embedding'] = [embeddings[node] for node in g.vs['name']]

In [3]:
labels = ['agreement', 'neutral', 'disagreement']
g = lcc_excluding_no_content()
g.vs['embedding'] = np.random.rand(g.vcount(), 10)
g.es['label'] = np.random.choice(labels, g.ecount())

In [4]:
# compute average distance between nodes of the same label
for label in labels:
    print(label)
    distances = []
    es = g.es.select(label_eq=label)
    for edge in es:
        u = g.vs[edge.source]
        v = g.vs[edge.target]
        distances.append(np.linalg.norm(u['embedding'] - v['embedding']))
    print(np.mean(distances))

agreement
1.2657159983586543
neutral
1.266620344486227
disagreement
1.266138573180785


In [5]:
g.es.select(label_eq='agreement')['color'] = 'green'
g.es.select(label_eq='neutral')['color'] = 'gray'
g.es.select(label_eq='disagreement')['color'] = 'red'

Paper?
* https://link.springer.com/article/10.1007/s10618-020-00716-6
* https://www.cs.unibo.it/~danilo.montesi/CBD/Articoli/SurveyComunityDectionMultilayerNetwork.pdf